### First attempt at building a Neural Network to learn a non-linear F(s)


In [20]:
import torch
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F

print(torch.__version__)

1.9.0


In [21]:
#Preprocess Data
df = pd.read_csv('../bricks_data/dataset_geometric.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268046 entries, 0 to 268045
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   lrg_density        268046 non-null  float64
 1   elg_density        268046 non-null  float64
 2   qso_density        268046 non-null  float64
 3   stellar_density    268046 non-null  float64
 4   airmass_galaxy     268046 non-null  float64
 5   fwhm_galaxy        268046 non-null  float64
 6   ebv_galaxy         268046 non-null  float64
 7   ccdnphotom_galaxy  268046 non-null  float64
 8   ccdskysb_galaxy_g  268046 non-null  float64
 9   ccdskysb_galaxy_r  268046 non-null  float64
 10  ccdskysb_galaxy_z  268046 non-null  float64
 11  exptime_galaxy_g   268046 non-null  float64
 12  exptime_galaxy_r   268046 non-null  float64
 13  exptime_galaxy_z   268046 non-null  float64
 14  meansky_galaxy_g   268046 non-null  float64
 15  meansky_galaxy_r   268046 non-null  float64
 16  me

### Defining The Dataset Class Inheriting from Torch.dataset to be able to use a dataloader for training

In [22]:
class DensitySurvey(Dataset):
    def __init__(self, path_to_data, galaxy_type):
        df = pd.read_csv(path_to_data)
        df = df[0:1000]
        # Extracting Targets and Input
        if galaxy_type == "LRG":
            self.target = df['lrg_density'].to_numpy(copy=True)
        if galaxy_type == "ELG":
            self.target = df['elg_density'].to_numpy(copy=True)
        if galaxy_type == "QSO":
            self.target = df['qso_density'].to_numpy(copy=True)
        self.input = df.drop(columns=['lrg_density','elg_density','qso_density']).to_numpy(copy=True)

        # Scaling
        scaler = preprocessing.MinMaxScaler()
        self.input = scaler.fit_transform(self.input, self.target)
        self.target = scaler.fit_transform(self.target)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return torch.from_numpy(self.input[idx]), torch.tensor(self.target[idx])

    def get_train_test(self):
        return train_test_split(self.input, self.target, test_size=0.33, random_state=44, shuffle=True)

In [23]:
data = DensitySurvey('../bricks_data/dataset_geometric.csv', 'LRG')




[-8.04639962e-01  7.91530233e-01  2.74687491e+00  5.89569954e-02
  2.18765625e+02  3.64787966e-01  3.55235636e-01  2.91855395e-01
  3.90625000e+00  2.73437500e+00  3.47039056e+00  4.87500522e-03
  1.13316346e-02  5.24217449e-02  3.69512200e-01  3.61478060e-01
  3.46347690e-01]
[3.92156863e-02 6.64966148e-02 1.74666158e-02 1.14171570e-01
 1.07725458e-02 9.03221215e-01 8.87659902e-01 4.06025945e-01
 1.00000000e+00 4.14243284e-01 8.34315856e-01 3.50922151e-03
 7.39596936e-04 5.12259635e-02 9.63289711e-01 9.66846871e-01
 8.02386365e-01]


In [16]:
x,y = data.__getitem__(3)

print(x,y)


tensor([-1.5344e-01,  1.7254e+00,  4.2657e+00,  2.1324e-01,  9.4662e+02,
         3.6582e-01,  3.3359e-01,  2.9586e-01,  3.3594e+00,  3.7500e+00,
         2.5094e+00,  5.0745e-03,  8.7749e-02,  4.0619e-02,  3.6639e-01,
         3.4505e-01,  3.4872e-01], dtype=torch.float64) tensor(0.4012, dtype=torch.float64)
